In [ ]:
!pip install flair
!python3 -m nltk.downloader punkt
!pip install spacy==2.1.0
!python3 -m spacy download en
!pip3 install neuralcoref

     |████████████████████████████████| 337kB 3.5MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 983kB 8.8MB/s 
     |████████████████████████████████| 1.1MB 20.7MB/s 
     |████████████████████████████████| 798kB 28.1MB/s 
     |████████████████████████████████| 19.7MB 162kB/s 
     |████████████████████████████████| 1.0MB 54.6MB/s 
     |████████████████████████████████| 276kB 43.4MB/s 
     |████████████████████████████████| 3.0MB 45.9MB/s 
     |████████████████████████████████| 890kB 59.7MB/s 
  Created wheel for segtok: filename=segtok-1.5.10-cp36-none-any.whl size=25021 sha256=f30b978bcdf98b7746fda7beb2431437a79c2c5ee943fb22baf3b69a09facbe7
  Stored in directory: /root/.cache/pip/wheels/b4/39/f6/9ca1c5cabde964d728023b5751c3a206a5c8cc40252321fb6b
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45612 sha256=259e331689b32c4bdd742a8242a54bdb7733de1b86fc291cd18c434b8ca3a5fc
  Stored in directory: /root/.cache/pip/

In [ ]:
from google.colab import drive
import glob
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")
import logging
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import random

class DisableLogger():
    def __enter__(self):
       logging.disable(logging.CRITICAL)
    def __exit__(self, a, b, c):
       logging.disable(logging.NOTSET)


In [ ]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
dataset_root = "/content/gdrive/My Drive/CMU/School/NLP/data/Development_data"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import string  
def load_and_preprocess(filename, joined=True):
  raw = open(filename, 'r', encoding='utf8').read()
  lines = raw.split("\n")
  lines = [l for l in lines if len(l) > 0 and l[-1] in string.punctuation]
  return " ".join(lines) if joined else lines

In [ ]:
from flair.embeddings import FlairEmbeddings, RoBERTaEmbeddings
from flair.data import Sentence

from flair.models import SequenceTagger

from flair.embeddings import DocumentMeanEmbeddings

with DisableLogger():
  bert_embeddings = RoBERTaEmbeddings(pretrained_model_name_or_path="roberta-large", layers="0,1,2,3,4,5,6,7,8,9,10,11,12",
                                pooling_operation="first", use_scalar_mix=True)

  document_embeddings = DocumentMeanEmbeddings([bert_embeddings])

  tagger = SequenceTagger.load('ner')

100%|██████████| 432197603/432197603 [00:19<00:00, 22280169.29B/s]


In [ ]:
import spacy
import neuralcoref
nlp = spacy.load("en")
_ = neuralcoref.add_to_pipe(nlp)

100%|██████████| 40155833/40155833 [00:00<00:00, 56561667.80B/s]


In [ ]:
doc_text = load_and_preprocess('/content/gdrive/My Drive/CMU/School/NLP/data/Development_data/set3/a2.txt')
query = "What did Chris and Ryan Kitteridge do when their son was a prisoner in the house of Masters of the Hall north of Saffron City?"

In [ ]:
query_doc = nlp(query)
combined_text = doc_text + "\n" + query
combined_doc = nlp(combined_text)
from nltk.tokenize import sent_tokenize
sent_text = sent_tokenize(doc_text)

In [ ]:
import re

from spacy.matcher import Matcher

def get_corefd_sentence(s):
  matcher = Matcher(nlp.vocab)
  s_doc = nlp(s)
  pattern = [{"TEXT": token.text} for token in s_doc]
  matcher.add("SMatch", None, pattern)
  matches = matcher(combined_doc)
  for match_id, start, end in matches:
    span = combined_doc[start:end]  
    return span
  return None

def dep_sentence(s):
  lookup_s = get_corefd_sentence(s)
  if lookup_s is None:
    lookup_s = nlp(s)
  deps = []
  for token in lookup_s:
    if token._.in_coref:
      key = token._.coref_clusters[0].main.text.lower()
    else:
      key = token.lemma_.lower()
    deps.append(("head", key, token.head.text.lower()))
    deps.append(("dep", key, token.dep_))
  return deps

def word_sentence_featurize(s):
  lookup_s = get_corefd_sentence(s)
  if lookup_s is None:
    lookup_s = nlp(s)
  
  tokens = []
  for token in lookup_s:
    if token.is_stop:
      continue
    else:
      if token._.in_coref:
        tokens.append(token._.coref_clusters[0].main.text.lower())
      else:
        tokens.append(token.lemma_.lower())
  return " ".join([t.lower() for t in tokens])

def dep_sentence_featurize(s):
  sentence_deps = dep_sentence(s)
  return " ".join([str(d).replace(" ", "_") for d in sentence_deps])

In [ ]:
sent_words = [word_sentence_featurize(s) for s in sent_text]

In [ ]:
cv_words = CountVectorizer(analyzer=lambda x : x.split(" "))
word_count_vector = cv_words.fit_transform(sent_words)
tfidf_transformer_words = TfidfTransformer(smooth_idf=True,use_idf=True)
_ = tfidf_transformer_words.fit(word_count_vector)

In [ ]:
sent_deps = [dep_sentence_featurize(s) for s in sent_text]

In [ ]:
cv_deps = CountVectorizer(analyzer=lambda x : x.split(" "))
dep_count_vector = cv_deps.fit_transform(sent_deps)
tfidf_transformer_deps = TfidfTransformer(smooth_idf=True,use_idf=True)
_ = tfidf_transformer_deps.fit(dep_count_vector)

In [ ]:
def embed_sentence(s):
  sentence = Sentence(s)
  document_embeddings.embed(sentence)
  return (s, sentence.get_embedding().cpu().data.numpy(), tfidf_transformer_words.transform(cv_words.transform([word_sentence_featurize(s)])).toarray().reshape(-1), tfidf_transformer_deps.transform(cv_deps.transform([dep_sentence_featurize(s)])).toarray().reshape(-1))

def cos_sim(p):
  a, b = p
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def compare_query(p):
  qv, sp = p
  similarity1 = cos_sim((qv[1], sp[1]))
  similarity2 = cos_sim((qv[2], sp[2]))
  similarity3 = cos_sim((qv[3], sp[3]))
  similarities = [similarity1, similarity2, similarity3]
  similarity = np.array(similarities).mean()
  return sp[0], similarity, similarities

In [ ]:
embedded_sentences = list(map(embed_sentence, sent_text))

In [ ]:
query_vec = embed_sentence(query)

In [ ]:
query_pairs = [(query_vec, e) for e in embedded_sentences]

In [ ]:
distances = list(map(compare_query, query_pairs))

In [ ]:
r = sorted(distances, key=lambda x : x[1])
distances_sorted = [(len(r) - 1 - i, r[i]) for i in range(len(r))]

In [ ]:
print(distances_sorted[::-1])

[(0, ('The portrayals of Ash in the manga The Electric Tale of Pikachu, and Ash & Pikachu, are very similar to the one in the anime.', 0.4438873299612654, [0.96964544, 0.32389450382034324, 0.038122045484992186])), (1, ("The Electric Tale of Pikachu is based on Ash's journey up until the end of the Orange Islands travel.", 0.43238092080831514, [0.9701172, 0.295171931197138, 0.03185361988594963])), (2, ('As the enduring comical villains of the series, the Team Rocket trio occasionally show signs of goodness, such as care and friendship with their Pokémon and each other, and despite their constant antagonism, they occasionally put their differences aside and work together with Ash against a common threat, such as in Pokémon: The Movie 2000, where they aid Ash in retrieving the three treasures in order to save the world.', 0.4135102626591083, [0.97043115, 0.20944765130208617, 0.0606519876693487])), (3, ('Bart remarks "Wow, how does this show stay so fresh?', 0.41186449488336724, [0.9615154

In [ ]:
best_response = distances_sorted[-1]

In [ ]:
doc_text

'Ash Ketchum, known as Satoshi (サトシ) in Japan, is a fictional character in the Pokémon franchise owned by Nintendo. He is the main protagonist of the Pokémon anime and manga series as well as on various merchandise related to the franchise. In Japanese, the character is voiced by Rica Matsumoto. In the English dub, he was voiced by Veronica Taylor from 1998 to 2006. Since the ninth season of the anime series, he is voiced by Sarah Natochenny for the remainder of the anime. His name is derived from the Japanese name (as the letters \'ash\' are included in \'Satoshi\') and his English motto, "Gotta Catch \'Em All". Ash\'s dream is to become a Pokémon Master. He is loosely based on Red, the player character from the Generation I games Pokémon Red, Green, Blue and Yellow as well as the Generation III games Pokémon FireRed and LeafGreen versions. The character\'s first official appearance in a game was in Pokémon Puzzle League. As the protagonist of the Pokémon anime, Ash has appeared in al

In [ ]:
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel

DEVICE = "cuda"

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')


# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model = model.eval()

model = model.to(DEVICE)


In [ ]:
def generate_question():
  NUM_SENTENCES = 24
  start_index = int(random.random() * (len(sent_text) - NUM_SENTENCES))

  starters = ["What", "When", "Where", "Who", "To whom", "Which", "Whose", "Why", "How"]

  starter_index = int(random.random() * (len(starters)))

  generated = None

  while generated is None or generated[-1] != "?":
    sent_start = start_index
    sent_end = start_index + NUM_SENTENCES
    text = " ".join(sent_text[sent_start:sent_end]) + "\n" + starters[starter_index]
    # text = "In Japanese, the character is voiced by Rica Matsumoto. Who"
    start_len = len(sent_tokenize(text))
    while len(sent_tokenize(text)) <= start_len:
      indexed_tokens = tokenizer.encode(text)

      # Convert indexed tokens in a PyTorch tensor
      tokens_tensor = torch.tensor([indexed_tokens])
      tokens_tensor = tokens_tensor.to(DEVICE)

      # Predict all tokens
      with torch.no_grad():
          outputs = model(tokens_tensor)
          predictions = outputs[0]

      pred = predictions[0, -1, :]
      pred_exp = F.softmax(pred, dim=0)
      predicted_index = torch.multinomial(pred_exp, 1).item()
      predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
      text = predicted_text

    gen_sents = sent_tokenize(text)
    new_generated = gen_sents[len(gen_sents) - 2]
    if new_generated == generated:
      break
    else:
      generated = new_generated
  return generated
  

In [ ]:
generate_question()

'Which first top-ten American game of all-time, Game or is it the first?'